In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import random

In [2]:
database = pd.read_csv('../../data/1930_census/Gotland/Folkräkning_1930_Gotlands_län', '\t')

/home/erik/Riksarkivet/Projects/handwritten-text-recognition/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (21,41,48,62,63,67,76,81) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def filter_df(df, batch):

    for name, cat_col in df.iteritems():

        for i, cat in enumerate(cat_col):

            row = i + 1

            res = database.query('BILDID==@name & RAD==@row')

            if (len(res) == 1 and int(res['FODAR'].item()) > 1911):
                df.at[i, name] = 0

    return df


In [4]:
def write_dataset(filtered_df, composition_dict, ds_path):


    written = {
        'clean': 0,
        'overwritten': 0,
        'noise_under': 0,
        'lines': 0,
        'noise_left': 0,
        'noise_over': 0,
        'text_over': 0,
        'stained': 0,
        'noise_right': 0,
        'faint_ink': 0
    }

    noisy_imgs = []
    clean_imgs = []

    for name, col in filtered_df.iteritems():
        
        for i, noise_cat in enumerate(col):
            
            paths = {
                'read': '',
                'write': ''
            }
            
            paths['read'] = os.path.join('../../data/1930_census/Gotland/cyclegan/raw_data', name, 'name', str(i + 1) + '.jpg')
            
            if noise_cat == 0:
                continue
            elif noise_cat == 1 and written['clean'] < composition_dict['clean']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                clean_imgs.append(paths)
                written['clean'] += 1
            elif noise_cat == 2 and written['overwritten'] < composition_dict['overwritten']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'     
                noisy_imgs.append(paths)
                written['overwritten'] += 1
            elif noise_cat == 3 and written['noise_under'] < composition_dict['noise_under']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)
            elif noise_cat == 4 and written['lines'] < composition_dict['lines']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)
            elif noise_cat == 5 and written['noise_left'] < composition_dict['noise_left']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)
            elif noise_cat == 6 and written['noise_over'] < composition_dict['noise_over']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)
            elif noise_cat == 7 and written['text_over'] < composition_dict['text_over']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)
            elif noise_cat == 8 and written['stained'] < composition_dict['stained']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)
            elif noise_cat == 9 and written['noise_right'] < composition_dict['noise_right']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)
            elif noise_cat == 10 and written['faint_ink'] < composition_dict['faint_ink']:
                paths['write'] = name + '_' + str(i + 1) + '.jpg'
                noisy_imgs.append(paths)

    random.shuffle(clean_imgs)
    random.shuffle(noisy_imgs)
    print(len(clean_imgs))
    print(len(noisy_imgs))
    
    for i, p in enumerate(zip(clean_imgs, noisy_imgs)):
        
        if i < 7:
            
            write_path_clean = os.path.join(ds_path, 'testB', p[0]['write'])
            write_path_noisy = os.path.join(ds_path, 'testA', p[1]['write'])
            
            img_clean = cv2.imread(p[0]['read'], cv2.IMREAD_GRAYSCALE)
            #ret, img_clean_bin = cv2.threshold(img_clean,150,255,cv2.THRESH_BINARY)

            img_noisy = cv2.imread(p[1]['read'], cv2.IMREAD_GRAYSCALE)

            
            cv2.imwrite(write_path_clean, img_clean)
            cv2.imwrite(write_path_noisy, img_noisy)

            print('hej')
            

        else:

            write_path_clean = os.path.join(ds_path, 'trainB', p[0]['write'])
            write_path_noisy = os.path.join(ds_path, 'trainA', p[1]['write'])

            img_clean = cv2.imread(p[0]['read'], cv2.IMREAD_GRAYSCALE)
            #ret, img_clean_bin = cv2.threshold(img_clean,150,255,cv2.THRESH_BINARY)

            img_noisy = cv2.imread(p[1]['read'], cv2.IMREAD_GRAYSCALE)

            
            try:
                cv2.imwrite(write_path_clean, img_clean)
                cv2.imwrite(write_path_noisy, img_noisy)
            except:
                print(p[1]['read'])
            
            #write noisy and clean to testA, testB

        #write rest to trainA, trainB

    

In [5]:
df = pd.read_excel('../../data/1930_census/Gotland/cyclegan/cycleGAN_ds_B0001097_and_B0001480.xlsx', header=0)

#df = filter_df(df, 'B0001097')



In [6]:
composition = {
    'clean': 641,
    'overwritten': 641,
    'noise_under': 0,
    'lines': 0,
    'noise_left': 0,
    'noise_over': 0,
    'text_over': 0,
    'stained': 0,
    'noise_right': 0,
    'faint_ink': 0
}

x = write_dataset(df, composition, '../../data/1930_census/Gotland/cyclegan/cyclegan_datasets/only_overwritten')

641
641
hej
hej
hej
hej
hej
hej
hej


In [19]:
#done with filter df, now check how many clean I got, and then continue with note

cat_count = {
    'no_cat': 0,
    'clean': 0,
    'overwritten': 0,
    'noise_under': 0,
    'lines': 0,
    'noise_left': 0,
    'noise_over': 0,
    'text_over': 0,
    'stained': 0,
    'noise_right': 0,
    'faint_ink': 0
}


for name, cat_col in df.iteritems():
    for cat in cat_col:
        if cat == 0:
            cat_count['no_cat'] += 1
        elif cat == 1:
            cat_count['clean'] += 1
        elif cat == 2:
            cat_count['overwritten'] += 1
        elif cat == 3:
            cat_count['noise_under'] += 1
        elif cat == 4:
            cat_count['lines'] += 1
        elif cat == 5:
            cat_count['noise_left'] += 1
        elif cat == 6:
            cat_count['noise_over'] += 1
        elif cat == 7:
            cat_count['text_over'] += 1
        elif cat == 8:
            cat_count['stained'] += 1
        elif cat == 9:
            cat_count['noise_right'] += 1
        elif cat == 10:
            cat_count['faint_ink'] += 1    






In [11]:
epoch = 830
s = 'epoch%d' % epoch

In [12]:
s

'epoch830'